In [28]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install -U langchain-community faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import OpenAIEmbeddings

In [21]:
import os
from dotenv import load_dotenv

load_dotenv('OpenAI.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
path_pkl = 'my_faiss_index/index.pkl'
path_faiss = 'my_faiss_index/index.faiss'
folder = 'my_faiss_index'

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import pickle




In [22]:
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.load_local("my_faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Medical assistant. Use the following context to answer the user's question. If you don't know the answer, say you don't know."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

In [24]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [32]:
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
)

ValidationError: 1 validation error for StuffDocumentsChain
  Value error, document_variable_name context was not found in llm_chain input_variables: ['chat_history', 'question'] [type=value_error, input_value={'llm_chain': LLMChain(ve...None, 'callbacks': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [26]:
print("Chatbot ready! Type 'exit' to quit.")
while True:
    question = input("You: ")
    if question.lower() in ["exit", "quit"]:
        break

    result = chain({"question": question})
    print("Bot:", result["answer"])

Chatbot ready! Type 'exit' to quit.


NameError: name 'chain' is not defined